<a href="https://colab.research.google.com/github/lady-ada/CIFAR10/blob/master/CIFAR10_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms

In [2]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
dataiter = iter(trainloader)
images, labels=dataiter.next()

print(len(dataiter))
print(images.shape)
print(labels.shape)
print(images[0].shape)
print(labels[0].item())

391
torch.Size([128, 3, 32, 32])
torch.Size([128])
torch.Size([3, 32, 32])
8


In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
class LeNet(nn.Module):

  def __init__(self):
    super(LeNet, self).__init__()
    self.cnn = nn.Sequential(
        nn.Conv2d(3, 6, 5),  # (N,1,32,32) ->  (N,6,28,28)
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2), # (N,6,28,28) -> (N,6,14,14)
        nn.ReLU(),
        nn.Conv2d(6, 16, 5), # (N,6,14,14) -> (N,16,10,10)
        nn.ReLU(),
        nn.AvgPool2d(2, stride=2) # (N,16,10,10) -> (N,16,5,5)
    )
    self.fc = nn.Sequential(
        nn.Linear(400, 120),
        nn.ReLU(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84, 10)
    )

  def forward(self, x):
    x = self.cnn(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:

def fit(trainloader, testloader, model, opt, loss_fn, epochs=16):

 
  for epoch in range(epochs):
    for i,data in enumerate(trainloader, 0):

      X, Y = data
      X, Y = X.to(device), Y.to(device)
      opt.zero_grad()

      Y_pred = model(X)
      loss = loss_fn(Y_pred, Y)
      loss.backward()
      opt.step()

    
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (epoch, 16, evaluation(testloader), evaluation(trainloader)))
 
  



In [0]:
def evaluation(dataloader):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [14]:
%%time
net = LeNet().to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), weight_decay=1e-5)
fit(trainloader ,testloader, net, opt, loss_fn)

Epoch: 0/16, Test acc: 36.07, Train acc: 35.35
Epoch: 1/16, Test acc: 42.87, Train acc: 42.68
Epoch: 2/16, Test acc: 47.07, Train acc: 47.86
Epoch: 3/16, Test acc: 49.17, Train acc: 50.04
Epoch: 4/16, Test acc: 51.48, Train acc: 52.57
Epoch: 5/16, Test acc: 51.81, Train acc: 53.16
Epoch: 6/16, Test acc: 53.63, Train acc: 55.40
Epoch: 7/16, Test acc: 54.21, Train acc: 56.27
Epoch: 8/16, Test acc: 54.57, Train acc: 56.99
Epoch: 9/16, Test acc: 55.12, Train acc: 57.58
Epoch: 10/16, Test acc: 56.38, Train acc: 59.41
Epoch: 11/16, Test acc: 56.69, Train acc: 59.92
Epoch: 12/16, Test acc: 57.53, Train acc: 60.92
Epoch: 13/16, Test acc: 58.54, Train acc: 62.15
Epoch: 14/16, Test acc: 57.12, Train acc: 61.66
Epoch: 15/16, Test acc: 59.48, Train acc: 64.07
CPU times: user 3min 3s, sys: 1.07 s, total: 3min 5s
Wall time: 3min 5s
